In [ ]:
import numpy as np
from os import listdir
from os.path import isfile, join
# from google.colab import files
# from google.colab import drive
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from google.colab.patches import cv2_imshow

import tensorflow as tf
import keras
from keras.datasets import mnist
import itertools 
from keras.utils import np_utils
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from sklearn.metrics import confusion_matrix 
from keras import layers
from tensorflow.keras.utils import to_categorical
#from keras.layers import Dense, Dropout , Input , Flatten , Conv2D , MaxPooling2D, BatchNormalization, Activation
from keras.callbacks import EarlyStopping , ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.image import img_to_array, load_img


In [ ]:
#used in blob detector
def extract_image_center(img, hcrop=0, wcrop=10):
    x = img.shape[0]
    y = img.shape[1]
    xc = int(hcrop*x/100)
    yc = int(wcrop*y/100)
    img = img[xc:x-xc, yc:y-yc]
    img = cv2.resize(img,(x,y))
    return img

In [ ]:
#CREATES ROWS FOR EXCEL SHEET.
def addSheetRow(sheet, row, ID, startX, startY, endX, endY, occurences, avgRadius):
  sheet.write(row,0, str(ID))
  sheet.write(row,1, f'{startX:.1f}' + ", " + f'{startY:.1f}' )
  sheet.write(row,2, f'{endX:.1f}' + ", " + f'{endY:.1f}' )
  sheet.write(row,3, '%d' % occurences )
  sheet.write(row,4, f'{avgRadius:.3f}' ) #make output limited to 3 decimals

In [ ]:
def generateAnalysisVideo(img_dir, video_dir, total_images):

  #get path to image directory
  image_directory = [os.path.join(img_dir,f) for f in os.listdir(img_dir)]

  fourcc = cv2.VideoWriter_fourcc(*'mp4v') #codec format
  video_file = video_dir + "video.mp4"
  #get path to video directory
  #video_directory = [os.path.join(video_file,f) for f in os.listdir(video_file)]


  img = []
  for img_name in image_directory:
    img.append(cv2.imread(img_name))

  height,width,layers=img[1].shape
  video=cv2.VideoWriter(video_file,fourcc,30,(width,height))

  for j in range(0,total_image_count): #create video
      video.write(img[j])

  cv2.destroyAllWindows()
  video.release()

In [ ]:
def process_image(image, target_shape):
    """Given an image, process it and return the array."""
    # Load the image.
    h, w, _ = target_shape
    image = load_img(image, target_size=(h, w))

    # Turn it into numpy, normalize and return.
    img_arr = img_to_array(image)
    #x = (img_arr / 255.).astype(np.float32)
    x = img_arr

    return x

In [ ]:
#This function will plot images in the form of a grid with 1 row and 10 columns 
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(100,100))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Making predictions with best CNN Model

In [ ]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/cse485/CNN_model_best.h5') #model_EcoFilter_Best.h5
#model.compile()

In [ ]:
test_image = process_image("/content/drive/MyDrive/cse485/data/test02/dirty/20210226_143748.jpg",( 150,150, 3)) #height,width, color channels
#test_image = cv2.imread(test_image, cv2.IMREAD_GRAYSCALE)
#test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)  ##turn image gray
#['class 0(Dirty)', 'class 1(Clean)']
cv2_imshow(test_image)
test_image = np.expand_dims(test_image, axis=0)  #keras expects extra batch dimension, so doing this line turns (150,150,3) into (None, 150, 150, 3), which is what the input layer of the model expects

In [ ]:
#testing on single image
test_prediction = model.predict(x=test_image,batch_size=10,verbose=0)
print(test_prediction)

[[0.00308549 0.95865476]]


In [ ]:
#model.compile()
#note: fixed error by looking at first layer shape and using that in ImageDataGenerator
model.summary()

In [ ]:
test_path = "/content/drive/MyDrive/cse485/data/test02"
#test_path = "/content/drive/MyDrive/Magic Zoom & Foldscope Images/E. coli/Before Ecofiltro/Optical Microscope (100X)"
#test_dir = "/content/drive/MyDrive/Magic Zoom & Foldscope Images/E. coli/Before Ecofiltro"
#test_class = "Optical Microscope (100X)"

In [ ]:
#NOTE: ImageDataGenerator requires the classes to be in their own subfolders from the parent directory or else it won't work. See https://kylewbanks.com/blog/loading-unlabeled-images-with-imagedatagenerator-flowfromdirectory-keras
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory = test_path, target_size=(150,150), classes=["clean","dirty"], batch_size=5, shuffle= False)

Found 133 images belonging to 2 classes.


In [ ]:
#predictions = model.predict(x = null, verbose=0)

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches,verbose=0)

In [ ]:
np.round(predictions)

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions,axis=-1))

In [ ]:
test_batches.class_indices

{'clean': 0, 'dirty': 1}

In [ ]:
cm_plot_labels = ['clean','dirty']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

# Import video and track blobs

In [ ]:
vidcap = cv2.VideoCapture('/content/drive/MyDrive/cse485/data/test_videos/good00.mp4')
success,image = vidcap.read()
count = 0

while success:
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  if success == True and count % 5 == 0:
    cv2_imshow(image)
  count += 1


# Video Method 01

In [ ]:
#Generate images from video and save them to directory
#generated_images_dir = "/content/drive/MyDrive/cse485/data/test03/"  #store frames from videos
generated_images_dir = "/content/drive/MyDrive/cse485/data/test03_v2/"  #store frames from videos
output_dir_imgs = "/content/drive/MyDrive/cse485/data/test_videos_output/images/" #output analysis images
output_dir_vid = "/content/drive/MyDrive/cse485/data/test_videos_output/video/" #output analysis video

In [ ]:
#PUT IMAGES FROM VIDEO INTO DIRECTORY

#video = cv2.VideoCapture('/content/drive/MyDrive/cse485/data/test_videos/good00.mp4')
video = cv2.VideoCapture('/content/drive/MyDrive/cse485/data/test_videos/real_good.mp4')
success,image = video.read()
count = 0

while success:
  dir = generated_images_dir + "frame%d.jpg" #path to save image to
  cv2.imwrite( dir % count, image)     # save frame      
  success,image = video.read()
  #print('Read a new frame: ', success)

  if success == True and count % 5 == 0: #if successfully gotten frame, save image to directory
    print('Read a new frame: ', success)
    #cv2_imshow(image)
    #shutil.move(image,'/content/drive/MyDrive/cse485/data/test03')     
  count += 1

In [ ]:
# SET UP BLOB DETECTOR
params = cv2.SimpleBlobDetector_Params() 

params.filterByInertia = False
params.filterByColor = False
#params.minThreshold = 10;
#params.minThreshold = 10;
  
# Set Area filtering parameters 
params.filterByArea = True
params.minArea = 5 #prev 10
params.maxArea = 55 #25


  
# Set Circularity filtering parameters 
params.filterByCircularity = True
params.minCircularity = 0.1
  
# Set Convexity filtering parameters 
params.filterByConvexity = True
params.minConvexity = 0.2 #0.2
      

#create blob detector
detector = cv2.SimpleBlobDetector_create(params)

In [ ]:
##TESTING IMAGE CROPS
img_name = "/content/drive/MyDrive/cse485/data/test03_v2/frame0.jpg"
og_img = cv2.imread(img_name, 1)                                                 #     img = img[center[0]-500:center[0]+500,center[1]-500:center[1]+500, :]
og_img = cv2.resize(og_img,(750,750))
keypoints = detector.detect(og_img)

print("Keypoints detected:" + str(len(keypoints)))

x,y = keypoints[0].pt
x, y = int(x), int(y)
size = int(keypoints[0].size)
sizeHalf = int(size/2)

print("size of keypoint 0: " + str(size))
print("X: " + str(x) + ", Y:" + str(y))
print("Image with keypoint 0")
im_with_keypoints = cv2.circle(og_img.copy(), (int(x),int(y)) , size, (0,243,255), 1 )
cv2_imshow(im_with_keypoints)


#print("Image with keypoint 0")
#cv2_imshow(og_img)
cropped = og_img[y - size:y + size, x - size:x + size]  #[y1:y2,x1:x2]

cropped = cv2.resize(cropped, (100,100))
cv2_imshow(cropped)

In [ ]:
#METHOD 01: Get keypoints every frame using blob detector

from keras.backend import maximum
import os
import math
from PIL.Image import NONE

import xlwt
from xlwt import Workbook



#the frames per second of the video
fps = 30 

frame_index = 0
color = 0

#total number of images in directory
total_image_count =  len(os.listdir(generated_images_dir))
max = total_image_count

#get path to image directory
image_directory = [os.path.join(generated_images_dir,f) for f in os.listdir(generated_images_dir)]

#whether to draw the keypoint indexs on the image
draw_kp_index = False

previous_keypoints = []

track_id = 0
tracking_objects = {} #store all tracked blobs
min_dist = 15 #minimum distance of keypoint to consider a blob as same blob

# Create spreadsheet to store output data
wb = Workbook()
outputSheet = wb.add_sheet('Sheet 1', cell_overwrite_ok=True)
row = 0
outputSheet.write(row, 0, 'Blob ID')
outputSheet.write(row, 1, 'Start Position (X,Y)')
outputSheet.write(row, 2, 'End Position (X,Y)')
outputSheet.write(row, 3, 'Occurences')
outputSheet.write(row, 4, 'Average Diameter')
row += 1

for img_name in image_directory:

  if frame_index < max:
                                                                                          
    #READ IN IMAGE AND RESIZE TO 750x750                                                  
    img = cv2.imread(img_name, 1)                                                 
    img = cv2.resize(img,(750,750))

    #GET CENTER OF IMAGE
    img_c = extract_image_center(img.copy(),  hcrop=40/2, wcrop=40/2) 
 
    #set image that we will draw on
    im_with_keypoints = img_c
    
    #get image keypoints using blob detector defined from above
    keypoints = detector.detect(img_c)

    if len(previous_keypoints) > 0:

      if len(tracking_objects) == 0: ##if no tracked objects currently

        keypoints_copy = keypoints.copy() # we cant update when looping, so we need a copy

        for kp in keypoints_copy:
            for pkp in previous_keypoints:

              x1, y1 = kp.pt
              x2,y2 = pkp.pt
              distance = math.hypot(x2 - x1, y2 - y1)

              #WHEN DISTANCE IS < MINIMUM, WE CONSIDER THAT AS THE SAME BLOB, SO WE REMOVE THAT KEYPOINT AND UPDATE THE PREVIOUS KEYPOINT POSITION
              if distance < min_dist:          
                keypoints.remove(kp)
                continue

        for kp in keypoints: ##add any remaining keypoints to tracked objects
           
              tracking_objects[track_id] = (kp.pt[0], kp.pt[1], kp.size, 1, kp.pt[0], kp.pt[1], kp.size, kp.size)  # ADD TRACKED ELEMENT ( X, Y , KEYPOINT RADIUS, OCCURENCE COUNT, STARTX, STARTY , SUM RADIUS, AVERAGE DIAMETER)
              #print("Added ID: %d , Distance between points (%d,%d) and (%d,%d) is: %d" % (track_id, x1,y1,x2,y2 ,distance) )
              track_id += 1

      elif len(tracking_objects) > 0:

        tracking_objects_copy = tracking_objects.copy()
        keypoints_copy = keypoints.copy()

        for object_id, tracked_kp_point in tracking_objects_copy.items():

            object_exists = False #check if saved tracked objects exits. If the distance is small, we can assume that the object exists still

            for kp in keypoints_copy: #kp refers to the NEWLY detected keypoints for the frame

              x1, y1 = kp.pt #TODO might need to do pt[0] and pt[1] &&&&&&&&&&&&&&&&&&&&&&&
              x2,y2 = tracked_kp_point[0], tracked_kp_point[1]
              distance = math.hypot(x2 - x1, y2 - y1)

              #WHEN DISTANCE IS < MINIMUM, WE CONSIDER THAT AS THE SAME BLOB, SO WE REMOVE THAT KEYPOINT AND UPDATE THE PREVIOUS KEYPOINT POSITION
              if distance < min_dist:   

                object_exists = True
                tracking_objects[object_id] = (kp.pt[0], kp.pt[1], kp.size, tracked_kp_point[3] + 1, tracked_kp_point[4], tracked_kp_point[5], tracked_kp_point[6] + kp.size, (tracked_kp_point[6] + kp.size) / (tracked_kp_point[3] + 1)  ) ## Update old point with new point (update position and increment occurence count)

                if kp in keypoints: #remove keypoint with small distanc
                  keypoints.remove(kp)

                continue
            
            #REMOVE KEYPOINTS THAT WERE LOST/NOT FOUND DURING THE CURRENT FRAME
            if not object_exists:
              addSheetRow(outputSheet,row,object_id,tracked_kp_point[4],tracked_kp_point[5],tracked_kp_point[0],tracked_kp_point[1],tracked_kp_point[3], tracked_kp_point[7]) #add row to sheet
              row += 1
              tracking_objects.pop(object_id)


        for kp in keypoints: ##add any remaining keypoints to tracked objects

          tracking_objects[track_id] = (kp.pt[0], kp.pt[1], kp.size, 1, kp.pt[0], kp.pt[1], kp.size, kp.size) # ADD TRACKED ELEMENT ( X, Y, KEYPOINT RADIUS, OCCURENCE COUNT, STARTX, STARTY , SUM RADIUS, AVERAGE DIAMETER)
          #print("Added ID: %d , Distance between points (%d,%d) and (%d,%d) is: %d" % (track_id, x1,y1,x2,y2 ,distance) )
          track_id += 1
      
        #REMOVE DUPLICATE KEYPOINTS (SAME X,Y COORDS)
        tracking_objects_copy = tracking_objects.copy()
        for object_id, tracked_kp_point in tracking_objects_copy.items():
          for object_id2, tracked_kp_point2 in tracking_objects_copy.items():
              x1, y1 = int(tracked_kp_point[0]), int(tracked_kp_point[1])
              x2,y2 = int(tracked_kp_point2[0]), int(tracked_kp_point2[1])

              if x1 == x2 and y1 == y2 and object_id != object_id2:

                if tracked_kp_point[3] >= tracked_kp_point2[3]: #remove duplicate that has least occurences
                  if object_id2 in tracking_objects.keys():
                    addSheetRow(outputSheet,row,object_id2,tracked_kp_point2[4],tracked_kp_point2[5],tracked_kp_point2[0],tracked_kp_point2[1],tracked_kp_point2[3], tracked_kp_point2[7]) #add row to sheet
                    row += 1
                    tracking_objects.pop(object_id2)
                else:
                  if object_id in tracking_objects.keys():
                    addSheetRow(outputSheet,row,object_id,tracked_kp_point[4],tracked_kp_point[5],tracked_kp_point[0],tracked_kp_point[1],tracked_kp_point[3], tracked_kp_point[7]) #add row to sheet
                    row += 1
                    tracking_objects.pop(object_id) 
                
      


        print("Tracked Blobs")
        print(tracking_objects)

    if frame_index >= max - 1: #WHEN WE ARE AT LAST FRAME ADD REMAINING BLOBS TO SPREADSHEET
      print("ended")
      for object_id, tracked_kp_point in tracking_objects.items():
        addSheetRow(outputSheet,row,object_id,tracked_kp_point[4],tracked_kp_point[5],tracked_kp_point[0],tracked_kp_point[1],tracked_kp_point[3], tracked_kp_point[7]) #add row to sheet
        row += 1

    #Draw the circle and text around keypoints
    for object_id, point in tracking_objects.items():
      x,y = point[0], point[1]
      radius = point[2] #size of keypoint area
      im_with_keypoints = cv2.circle(im_with_keypoints, (int(x),int(y)) , int(radius), (0,243,255), 1 )
      im_with_keypoints = cv2.putText(im_with_keypoints, str(object_id),(int(x),int(y - 7)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1 )

    
    #SHOW IMAGE
    cv2_imshow( im_with_keypoints)

    #make copy of points
    previous_keypoints = keypoints.copy()

    frame_index += 1

    cv2.waitKey(1000)
    cv2.destroyAllWindows()

    #SAVE OUTPUT ANALYSIS IMAGES
      #dir = output_dir_imgs + "frame%d.png" 
      #cv2.imwrite( dir % frame_index, im_with_keypoints) 

    print("\n========================\n")

#CREATE OUTPUT SPREADHSEET
wb.save('/content/drive/MyDrive/cse485/data/output.xls')

#CREATE ANALYSIS VIDEO
  #generateAnalysisVideo(output_dir_imgs, output_dir_vid, total_image_count) 



              #keypoints[0]
       # if cv2.KeyPoint_overlap(previous_keypoints[0] , keypoints[0])== 1:
         # print("OVERLAP")
           #im_with_keypoints = cv2.putText(im_with_keypoints, "kp_%d (%d,%d)" % (kp_index, int(x), int(y)),(int(x),int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,234,255), 1 )
           # #The flag DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob. See https://docs.opencv.org/3.4/de/d30/structcv_1_1DrawMatchesFlags.html
            #cv2.drawKeypoints(img_c, keypoints, im_with_keypoints, (0,0,color), cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG,)  #draws over previous image
          #im_with_keypoints = cv2.drawKeypoints(img_c, keypoints, np.array([]), (0,0,color), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) # original
           # im_with_keypoints = cv2.drawKeypoints(im_with_keypoints, all_kp, np.array([]), (0,243,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) # original


In [ ]:
def addSheetRow(sheet, row, ID, startX, startY, endX, endY, occurences):
  sheet.write(row,0, str(ID))
  sheet.write(row,1, '(%d,%d)' % ( startX , startY  ) )
  sheet.write(row,2, '(%d,%d)' % ( endX , endY  ) )
  sheet.write(row,3, '%d' % occurences )


In [ ]:
# Writing to an excel 
# sheet using Python
import xlwt
from xlwt import Workbook
  
# Workbook is created
wb = Workbook()

# add_sheet is used to create sheet.
outputSheet = wb.add_sheet('Sheet 1', cell_overwrite_ok=True)
row = 0

#(row, column)
outputSheet.write(row, 0, 'Blob ID')
outputSheet.write(row, 1, 'Start Position (X,Y)')
outputSheet.write(row, 2, 'End Position (X,Y)')
outputSheet.write(row, 3, 'Occurences')
row += 1

addSheetRow(outputSheet,row,1,25,25,50,50,100)
row += 1
addSheetRow(outputSheet,row,2,27,25,80,50,691)


  
wb.save('/content/drive/MyDrive/cse485/data/output.xls')

# Testing Optical Flow tracking

In [ ]:
#set up variables
lk_params = dict(winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict(maxCorners = 20,
                    qualityLevel = 0.7, #0.3 quality of tracking
                    minDistance = 1, #10
                    blockSize = 7 )


trajectory_len = 80
detect_interval = 1
trajectories = []
frame_idx = 0

# SET UP BLOB DETECTOR
params = cv2.SimpleBlobDetector_Params() 
  
# Set Area filtering parameters 
params.filterByArea = False
#params.minArea = 0.1 #prev 10
#params.maxArea = 10

params.filterByColor = False
  
# Set Circularity filtering parameters 
params.filterByCircularity = False
#params.minCircularity = 0.9
  
# Set Convexity filtering parameters 
params.filterByConvexity = False
#params.minConvexity = 0.2 #0.2
      
# Set inertia filtering parameters 
params.filterByInertia = False
#params.minInertiaRatio = 0.01

#create blob detector
detector = cv2.SimpleBlobDetector_create(params)


In [ ]:
from keras.backend import maximum
import os
from PIL.Image import NONE


#total number of images in directory
total_image_count =  len(os.listdir(generated_images_dir))
max = total_image_count

#get path to image directory
image_directory = [os.path.join(generated_images_dir,f) for f in os.listdir(generated_images_dir)]
#the frames per second of the video
fps = 30 

frame_index = 0
color = 0
draw_path_trail = False #whether to show trail of movement
draw_bounding_box = False
draw_keypoints = False #whether to show the keypoints from the Blob detector

previous_keypoints = None
overlapped_keypoints = []

print("Getting images from:" + generated_images_dir + "\nTotal images: " + str(total_image_count) )

for img_name in image_directory:

  if frame_index < max:#total_image_count:

    #read in image and resize to 750x750                                                 
    img = cv2.imread(img_name, 1)                                                
    img = cv2.resize(img,(750,750))
    img_c = extract_image_center(img.copy(),  hcrop=40/2, wcrop=40/2)  #get center of image

    frame_gray = cv2.cvtColor(img_c, cv2.COLOR_BGR2GRAY) #optical flow image requires gray image
    img = img_c.copy()

#-------------Keypoints
    #set image that we will draw on
   # im_with_keypoints = img_c

    #get image keypoints using blob detector 
    #keypoints = detector.detect(img_c)

    #draw keypoints over image
   # im_with_keypoints = cv2.drawKeypoints(img_c, keypoints, np.array([]), (0,0,color), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) 
#-------------------------------

    # Calculate optical flow for a sparse feature set using the iterative Lucas-Kanade Method
    if len(trajectories) > 0: #if any trajectories that were tracked or 

        img0, img1 = prev_gray, frame_gray # get previous frame and current frame
        p0 = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2) #create points
        p1, _st, _err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)
        p0r, _st, _err = cv2.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)

        diff = abs(p0-p0r).reshape(-1, 2).max(-1) #get differnce between points

        good = diff < 1 #if diff is less than 1 the optical flow is good

        new_trajectories = []

        # Get all the trajectories
        for trajectory, (x, y), good_flag in zip(trajectories, p1.reshape(-1, 2), good):
           
            if not good_flag:
                continue

            trajectory.append((x, y)) #keep updating trajectory as we detect new points

            if len(trajectory) > trajectory_len: #limit number of points displayed
                del trajectory[0]

            new_trajectories.append(trajectory) #add new trajectory

            # Newest detected point
            cv2.circle(img, (int(x), int(y)), 2, (0, 0, 255), -1) #creates red point that shows the closest point between two frames
            #cv2.putText(img, "point_%d (%d,%d)" % (1, int(x), int(y)),(int(x),int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,234,255), 1 ) 
            #print("point_%d (%d,%d)" % (1, int(x), int(y)))

          
        trajectories = new_trajectories #update our trajectories list

        # Draw all the trajectories
        cv2.polylines(img, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0)) #creates the green lines that follows the red point, which shows trajectories
 
        #cv2.putText(img, 'track count: %d' % len(trajectories), (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)


    # Update interval - When to update and detect new features
    if frame_idx % detect_interval == 0:

        #Create two masks. mask1 is used for detecting good features in optimal flow, mask2 is used to generate bounding box around blob contours
        mask = np.zeros_like(frame_gray) #Return an array of zeros with the same shape and type as a given array
        mask[:] = 255 #turn image white
        mask2 = np.zeros_like(frame_gray) #Return an array of zeros with the same shape and type as a given array
        mask2[:] = 0 #turn image black

        t = 0
        z=[(0,0)]
        # Lastest point in latest trajectory
        for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:

          if not (x,y) in z:
            #print("Added point at " + str(x) + "," + str(y))
            cv2.circle(mask2, (x, y), 5, 255, -1) #draw white circles on MASK image
            #cv2.putText(img, "point_%d (%d,%d)" % (1, int(x), int(y)),(int(x),int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,234,255), 1 )
            z.append((x,y))
            z.append((x + 1, y - 1)) #trim out points that are too close
            z.append((x - 1, y + 1)) #trim out points that are too close
            z.append((x + 1, y)) #trim out points that are too close
            z.append((x - 1, y)) #trim out points that are too close
            z.append((x , y + 1)) #trim out points that are too close
            z.append((x , y - 1)) #trim out points that are too close
            z.append((x + 1,y + 1)) #trim out points that are too close
            z.append((x - 1,y - 1))
            t+=1

          #cv2.circle(mask2, (x, y), 5, 255, -1) #draw white circles on MASK image
         # z.append((x,y))
        
          #print("Blobs Detected: " + str(t))
          #Get countours for mask image, then draw bounding rectangle around countours
          contours, _ = cv2.findContours(mask2,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
          if contours:
            for cnt in contours:
              x1,y1,w1,h1 = cv2.boundingRect(cnt)
              cv2.rectangle(img, (x1,y1), (x1+w1, y1+h1), (0,0,255), 1) #draw bound box around movement area
      
              

        # Detect the good features to track
        p = cv2.goodFeaturesToTrack(frame_gray, mask = mask, **feature_params)
        if p is not None:
            # If good features can be tracked - add that to the trajectories
            for x, y in np.float32(p).reshape(-1, 2):
                trajectories.append([(x, y)])


    frame_idx += 1
    prev_gray = frame_gray

    if frame_idx > 0:#max - 12:
      #show analysis image
      if frame_idx > 149:
        #cv2_imshow(img) 
        print("")
    #save output analysis images
    dir = output_dir_imgs + "frame%d.png" 
    cv2.imwrite( dir % frame_idx, img)   



    #cv2_imshow(mask2)
    cv2.waitKey(100)
    cv2.destroyAllWindows()
    frame_index += 1

#generateAnalysisVideo(output_dir_imgs, output_dir_vid, total_image_count) #create video from the analysis images